In [ ]:
import os
import pandas as pd
import pyarrow as pa
print(pd.__version__, pa.__version__)

In [ ]:
## change working directory
os.chdir('/app')
## show current working directory
os.getcwd()

In [ ]:
import pyarrow.dataset as ds
import pyarrow.fs as fs
import pyarrow.parquet as pq
from ml.utils.spark_utils import get_spark_session

In [ ]:
# Create an S3-compatible filesystem for MinIO
s3 = fs.S3FileSystem(
    access_key="minioadmin",
    secret_key="minioadmin",
    endpoint_override="http://minio:9000",  # MinIO endpoint
)

In [ ]:
file_path = "recommendation-system/data/silver/netflix_user_data.parquet"
dataset = ds.dataset(file_path, filesystem=s3, format="parquet")

In [ ]:
pq.read_table(
    source=file_path,
    filesystem=s3
)

In [ ]:
for batch in dataset.to_batches(batch_size=100_000):
    df = batch.to_pandas()
    print(df.shape)
    print(df.dtypes)
    break

In [ ]:
spark = get_spark_session()

In [ ]:
df = spark.read.parquet(f"s3a://{file_path}")

In [ ]:
df.printSchema()

In [ ]:
df.show(20)

In [ ]:
df.select("MovieID").distinct().show(50)

In [ ]:
df.describe().show()

In [ ]:
spark.stop()